# Generate configuration using the `revisitpy` package

Source: https://revisit.dev/docs/revisitpy/
Also: 
- https://github.com/revisit-studies/revisitpy-examples
- https://github.com/revisit-studies/revisitpy

In [40]:
import revisitpy as rvt
import pandas as pd 

## Metadata 

- Example here: https://revisit.dev/docs/revisitpy/examples/example_jnd_study/ 

In [41]:
# Meta Data
study_metadata = rvt.studyMetadata(
    authors=["Brian Bollen"],
    organizations=["Visualization Design Lab"],
    title='Correlation Judgment Study',
    description='',
    date='2025-01-13',
    version='1.0'
)

# UI Config
ui_config = rvt.uiConfig(
  contactEmail="briancbollen@gmail.com",
  logoPath="./assets/revisitLogoSquare.svg",
  withSidebar=True,
  withProgressBar=False,
  nextOnEnter=True
)

print(study_metadata)
print(ui_config)

{
    "authors": [
        "Brian Bollen"
    ],
    "date": "2025-01-13",
    "description": "",
    "organizations": [
        "Visualization Design Lab"
    ],
    "title": "Correlation Judgment Study",
    "version": "1.0"
}
{
    "contactEmail": "briancbollen@gmail.com",
    "logoPath": "./assets/revisitLogoSquare.svg",
    "nextOnEnter": true,
    "withProgressBar": false,
    "withSidebar": true
}


## Generate component related 

In [42]:
response = rvt.response(
    id = "retrieve_value", 
    prompt = 'Your selected answer', 
    location = 'belowStimulus', 
    type = 'numerical', 
    required = True,
)

In [43]:
new_df = pd.DataFrame({"id": [i + 1 for i in range(10)]})
# print(new_df)
new_df.to_csv('data.csv', index=False)
# print(rvt.data("data.csv"))

data_sequence = rvt.sequence(order = 'random', numSamples=5).from_data(rvt.data("data.csv"))
print(data_sequence)

{
    "components": [
        "place-holder-component_id:1",
        "place-holder-component_id:2",
        "place-holder-component_id:3",
        "place-holder-component_id:4",
        "place-holder-component_id:5",
        "place-holder-component_id:6",
        "place-holder-component_id:7",
        "place-holder-component_id:8",
        "place-holder-component_id:9",
        "place-holder-component_id:10"
    ],
    "numSamples": 5.0,
    "order": "random"
}


In [44]:
def retrieve_value_component_function(id): 
    comp = rvt.component(
        component_name__ = f'retrieve_value_{id}',
        type = 'vega',
        response = [response],
        path = f'./specs/{id}.json', 
        instruction = f'Enter the answer to the question', 
        instructionLocation = 'belowStimulus',
        withSidebar = False,
    )
    return comp

In [39]:
data_sequence.component(retrieve_value_component_function)
# print(data_sequence)
print(data_sequence.get_components()[0])

AttributeError: 'VegaComponent' object has no attribute 'meta'

   id
0   1
1   2
2   3
3   4
4   5
5   6
6   7
7   8
8   9
9  10
